In [1]:
import numpy as np
import pandas as pd

from datetime import datetime
from time import sleep

import pickle

from pprint import pprint

from hab.airtable.airtable import AirtableHAB
from hab.serpapi import serpapi, locations
from hab.clean_functions import clean

In [2]:
today = datetime.now().strftime("%Y%m%d")
# print(today)

with open(file = "serpapi_key.txt", mode = "r") as file:
    api_key = file.read()
    
# print(api_key)

lista_puestos = pd.read_excel("Lista_Puestos.xlsx", header = None)

lista_puestos.columns = ["jobs"]

In [9]:
lista_puestos

,jobs
0,Desarrollador de software
1,Programador web
2,Consultor tecnológico
3,Ingeniero de sistemas
4,Especialista en seguridad informática
...,...
478,Desarrollador de software científico
479,Técnico de soporte remoto
480,Especialista en redes informáticas
481,Analista de seguridad en la nube


In [4]:
with open("lista_puestos_ingles.txt", "r") as file:
    lista_puestos_ingles = file.read()
    
lista_puestos_ingles = lista_puestos_ingles.split("\n")

lista_puestos_ingles

['Software Developer',
 'Web Programmer',
 'Technology Consultant',
 'Systems Engineer',
 'Information Security Specialist',
 'Programmer Analyst',
 'Mobile App Developer',
 'Database Administrator',
 'Software Architect',
 'Network Technician',
 'Cybersecurity Specialist',
 'IT Project Manager',
 'UX/UI Designer',
 'DevOps Engineer',
 'Support Technician',
 'Data Analyst',
 'Artificial Intelligence Specialist',
 'Video Game Developer',
 'QA Tester',
 'SAP Consultant',
 'Full-stack Developer',
 'Digital Graphic Designer',
 'Blockchain Specialist',
 'Data Scientist',
 'Software Engineer',
 'Big Data Consultant',
 'Systems Administrator',
 'Cloud Computing Specialist',
 'Information Security Analyst',
 'Java Developer',
 'Systems Technician',
 'Machine Learning Specialist',
 'Front-end Developer',
 'IoT Specialist (Internet of Things)',
 'Digital Transformation Consultant',
 '.NET Developer',
 'Virtual Reality Specialist',
 'Industrial Cybersecurity Specialist',
 'Telecommunications Tech

In [5]:
[x for x in [x.replace("Developer", "") for x in lista_puestos_ingles] if len(x.split()) > 1]

['Web Programmer',
 'Technology Consultant',
 'Systems Engineer',
 'Information Security Specialist',
 'Programmer Analyst',
 'Mobile App ',
 'Database Administrator',
 'Software Architect',
 'Network Technician',
 'Cybersecurity Specialist',
 'IT Project Manager',
 'UX/UI Designer',
 'DevOps Engineer',
 'Support Technician',
 'Data Analyst',
 'Artificial Intelligence Specialist',
 'Video Game ',
 'QA Tester',
 'SAP Consultant',
 'Digital Graphic Designer',
 'Blockchain Specialist',
 'Data Scientist',
 'Software Engineer',
 'Big Data Consultant',
 'Systems Administrator',
 'Cloud Computing Specialist',
 'Information Security Analyst',
 'Systems Technician',
 'Machine Learning Specialist',
 'IoT Specialist (Internet of Things)',
 'Digital Transformation Consultant',
 'Virtual Reality Specialist',
 'Industrial Cybersecurity Specialist',
 'Telecommunications Technician',
 'Security Consultant',
 'Deep Learning Specialist',
 'Data Mining Specialist',
 'Network Specialist',
 'Enterprise Arc

In [6]:
latam = [["com.ar", "Argentina"]         ,
         ["com.bo", "Bolivia"]           ,
         ["com.br", "Brazil"]            ,
         ["ca", "Canada"]            ,
         ["cl", "Chile"]             ,
         ["com.co", "Colombia"]          ,
         ["co.cr", "Costa Rica"]        ,
         ["com.cu", "Cuba"]              ,
         ["com.do", "Dominican Republic"],
         ["com.ec", "Ecuador"]           ,
         ["com.sv", "El Salvador"]       ,
         ["com.gt", "Guatemala"]         ,
#          ["gy", "Guyana"]            ,
#          ["ht", "Haiti"]             ,
         ["hn", "Honduras"]          ,
#          ["jm", "Jamaica"]           ,
         ["com.mx", "Mexico"]            ,
         ["com.ni", "Nicaragua"]         ,
         ["com.pa", "Panama"]            ,
         ["com.py", "Paraguay"]          ,
         ["com.pe", "Peru"]              ,
         ["com.pr", "Puerto Rico"]       ,
         ["com.uy", "Uruguay"]           ,
#          ["ve", "Venezuela"]
        ]

latam = [[".es", "Spain"]]

latam

[['.es', 'Spain']]

In [7]:
%%time

df = pd.DataFrame()

for abr, country in latam:
    
    for q in lista_puestos_ingles[::-1]:

        try:
            for pagination in range(100):
                
#                 sleep(0.3)

                print(f"{q:60}{pagination}")

                q_params = {"q"             : q,
                            "api_key"       : api_key,
                            "location"      : country.title(),
                            "start"         : pagination*10,
#                             "gl"            : abr,
#                             "hl"            : "es",
#                             "google_domain" : f"google.{abr}",
#                             "chips"         : "date_posted:year"
                            }

                response = serpapi.job_search(**q_params)


                if ("error" in response) or (response.get("jobs_results") == None) or (len(response.get("jobs_results")) < 10):
                    break
                
                now = datetime.now()
                
                with open(file = f"json_searches_latam/{str(now.timestamp()).replace('.', '')}.pkl", mode = "bw") as file:
                    pickle.dump(response, file)

                df_response = pd.json_normalize(response["jobs_results"])

                df_response.columns = [x.split(".")[0] if len(x.split(".")) == 1 else x.split(".")[-1] for x in df_response.columns]

                df_response["country_search"] = country.title()
            #     df_response = df_response[["job_id", "title", "company_name", "location", "source", "description", "country", "date_posted", "contract_type", "remote_work"]]

                df = pd.concat(objs = [df, df_response], ignore_index = True)
        except:
            print(f"Error {q} ***************************************************************")
            
        
            
            
df = df.drop_duplicates(subset = "job_id").reset_index(drop = True)

df.to_csv(f"extraccion_{today}_2.csv", index = False)
df.to_pickle(f"extraccion_{today}_2.pkl")

Educational Software Developer                              0
https://serpapi.com/search
Cloud Security Analyst                                      0
https://serpapi.com/search
Cloud Security Analyst                                      1
https://serpapi.com/search
Computer Networks Specialist                                0
https://serpapi.com/search
Remote Support Technician                                   0
https://serpapi.com/search
Scientific Software Developer                               0
https://serpapi.com/search
Quality Data Analyst                                        0
https://serpapi.com/search
Quality Data Analyst                                        1
https://serpapi.com/search
CRM Software Development Specialist                         0
https://serpapi.com/search
AS400 Systems Technician                                    0
https://serpapi.com/search
Hospital Information Systems Specialist                     0
https://serpapi.com/search
Mobile Solutions Deve

React Developer                                             5
https://serpapi.com/search
React Developer                                             6
https://serpapi.com/search
Quality Control Systems Specialist                          0
https://serpapi.com/search
Embedded Software Engineer                                  0
https://serpapi.com/search
Embedded Software Engineer                                  1
https://serpapi.com/search
Embedded Software Engineer                                  2
https://serpapi.com/search
Embedded Software Engineer                                  3
https://serpapi.com/search
Embedded Software Engineer                                  4
https://serpapi.com/search
Embedded Software Engineer                                  5
https://serpapi.com/search
Embedded Software Engineer                                  6
https://serpapi.com/search
Embedded Software Engineer                                  7
https://serpapi.com/search
Embedded Software Eng

User Experience Designer                                    5
https://serpapi.com/search
User Experience Designer                                    6
https://serpapi.com/search
User Experience Designer                                    7
https://serpapi.com/search
User Experience Designer                                    8
https://serpapi.com/search
User Experience Designer                                    9
https://serpapi.com/search
User Experience Designer                                    10
https://serpapi.com/search
User Experience Designer                                    11
https://serpapi.com/search
User Experience Designer                                    12
https://serpapi.com/search
Hardware Technician                                         0
https://serpapi.com/search
Hardware Technician                                         1
https://serpapi.com/search
Embedded Software Development Specialist                    0
https://serpapi.com/search
Embedded Software 

.NET Programmer                                             2
https://serpapi.com/search
Artificial Vision Systems Developer                         0
https://serpapi.com/search
Artificial Vision Systems Developer                         1
https://serpapi.com/search
Security Systems Technician                                 0
https://serpapi.com/search
Industrial Cybersecurity Consultant                         0
https://serpapi.com/search
Robotics Engineer                                           0
https://serpapi.com/search
Robotics Engineer                                           1
https://serpapi.com/search
Robotics Engineer                                           2
https://serpapi.com/search
Robotics Engineer                                           3
https://serpapi.com/search
Robotics Engineer                                           4
https://serpapi.com/search
Firmware Developer                                          0
https://serpapi.com/search
Firmware Developer   

ERP Implementation Specialist                               0
https://serpapi.com/search
Server Administration Specialist                            0
https://serpapi.com/search
Server Administration Specialist                            1
https://serpapi.com/search
Server Administration Specialist                            2
https://serpapi.com/search
Application Cybersecurity Specialist                        0
https://serpapi.com/search
Application Cybersecurity Specialist                        1
https://serpapi.com/search
Application Cybersecurity Specialist                        2
https://serpapi.com/search
Application Cybersecurity Specialist                        3
https://serpapi.com/search
Application Cybersecurity Specialist                        4
https://serpapi.com/search
Application Cybersecurity Specialist                        5
https://serpapi.com/search
Application Cybersecurity Specialist                        6
https://serpapi.com/search
Application Cybersecu

...

Linux Systems Administrator                                 0
https://serpapi.com/search
Linux Systems Administrator                                 1
https://serpapi.com/search
Linux Systems Administrator                                 2
https://serpapi.com/search
Business Intelligence Specialist                            0
https://serpapi.com/search
Business Intelligence Specialist                            1
https://serpapi.com/search
Business Intelligence Specialist                            2
https://serpapi.com/search
Business Intelligence Specialist                            3
https://serpapi.com/search
Business Intelligence Specialist                            4
https://serpapi.com/search
Business Intelligence Specialist                            5
https://serpapi.com/search
Business Intelligence Specialist                            6
https://serpapi.com/search
Business Intelligence Specialist                            7
https://serpapi.com/search
Virtualization Specia

Security Consultant                                         4
https://serpapi.com/search
Security Consultant                                         5
https://serpapi.com/search
Security Consultant                                         6
https://serpapi.com/search
Telecommunications Technician                               0
https://serpapi.com/search
Industrial Cybersecurity Specialist                         0
https://serpapi.com/search
Industrial Cybersecurity Specialist                         1
https://serpapi.com/search
Virtual Reality Specialist                                  0
https://serpapi.com/search
.NET Developer                                              0
https://serpapi.com/search
.NET Developer                                              1
https://serpapi.com/search
.NET Developer                                              2
https://serpapi.com/search
.NET Developer                                              3
https://serpapi.com/search
Digital Transformatio

Software Engineer                                           13
https://serpapi.com/search
Software Engineer                                           14
https://serpapi.com/search
Software Engineer                                           15
https://serpapi.com/search
Software Engineer                                           16
https://serpapi.com/search
Software Engineer                                           17
https://serpapi.com/search
Software Engineer                                           18
https://serpapi.com/search
Software Engineer                                           19
https://serpapi.com/search
Software Engineer                                           20
https://serpapi.com/search
Software Engineer                                           21
https://serpapi.com/search
Software Engineer                                           22
https://serpapi.com/search
Software Engineer                                           23
https://serpapi.com/search
Data Scien

Data Analyst                                                6
https://serpapi.com/search
Data Analyst                                                7
https://serpapi.com/search
Data Analyst                                                8
https://serpapi.com/search
Data Analyst                                                9
https://serpapi.com/search
Data Analyst                                                10
https://serpapi.com/search
Data Analyst                                                11
https://serpapi.com/search
Data Analyst                                                12
https://serpapi.com/search
Data Analyst                                                13
https://serpapi.com/search
Data Analyst                                                14
https://serpapi.com/search
Data Analyst                                                15
https://serpapi.com/search
Data Analyst                                                16
https://serpapi.com/search
Data Analyst  

Software Architect                                          4
https://serpapi.com/search
Software Architect                                          5
https://serpapi.com/search
Software Architect                                          6
https://serpapi.com/search
Software Architect                                          7
https://serpapi.com/search
Software Architect                                          8
https://serpapi.com/search
Database Administrator                                      0
https://serpapi.com/search
Database Administrator                                      1
https://serpapi.com/search
Database Administrator                                      2
https://serpapi.com/search
Database Administrator                                      3
https://serpapi.com/search
Database Administrator                                      4
https://serpapi.com/search
Database Administrator                                      5
https://serpapi.com/search
Database Administrato

In [11]:
response

{'search_metadata': {'id': '64f4cab1969c0887bef341d7',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/f8fe2dc31485b063/64f4cab1969c0887bef341d7.json',
  'created_at': '2023-09-03 18:04:33 UTC',
  'processed_at': '2023-09-03 18:04:33 UTC',
  'google_jobs_url': 'https://www.google.com/search?q=Software+Developer&ibp=htl;jobs&uule=w+CAIQICIFU3BhaW4&start=220',
  'raw_html_file': 'https://serpapi.com/searches/f8fe2dc31485b063/64f4cab1969c0887bef341d7.html',
  'total_time_taken': 1.65},
 'search_parameters': {'q': 'Software Developer',
  'engine': 'google_jobs',
  'location_requested': 'Spain',
  'location_used': 'Spain',
  'google_domain': 'google.com',
  'start': 220},
 'jobs_results': [{'title': 'Software Engineer - Industrial IoT',
   'company_name': 'Canonical - Jobs',
   'location': '  Madrid, Spain   ',
   'via': 'via Greenhouse',
   'description': 'This is an exciting opportunity for a software engineer passionate about open source software, Linux, edge comp

In [12]:
df_new = df.copy()
df_new.drop_duplicates(subset = "description", inplace = True)
df_new.reset_index(drop = True, inplace = True)

df_new.shape

(8455, 15)

In [13]:
df_new["date_posted"] = datetime.strptime("2023-08-02", "%Y-%m-%d").date()
df_new

,title,company_name,location,via,description,job_highlights,related_links,thumbnail,extensions,job_id,posted_at,schedule_type,work_from_home,country_search,salary,date_posted
0,Cloud Security Analyst,Azertium IT Global Services SL,"Madrid, Spain",via BeBee,Estamos ampliando el equipo En Azertium IT te ...,[{'items': ['Estamos ampliando el equipo En Az...,[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[13 hours ago, Full-time]",eyJqb2JfdGl0bGUiOiJDbG91ZCBTZWN1cml0eSBBbmFseX...,13 hours ago,Full-time,NaN,Spain,NaN,2023-08-02
1,Cyber Security Analyst,isolutions,"Barcelona, Spain",via Openings At Isolutions,Is Cyber Security your passion? More and more ...,[{'items': ['Is Cyber Security your passion? M...,[{'link': 'https://www.google.com/search?sca_e...,NaN,[Full-time],eyJqb2JfdGl0bGUiOiJDeWJlciBTZWN1cml0eSBBbmFseX...,NaN,Full-time,NaN,Spain,NaN,2023-08-02
2,Cloud & Cybersecurity Analyst I,BANCO SANTANDER S.A.,Spain,via Jobrapido.com,Cloud & Cybersecurity Analyst I\n\nCountry: Sp...,[{'items': ['Cloud & Cybersecurity Analyst I ...,"[{'link': 'http://www.santander.com/', 'text':...",NaN,[Full-time],eyJqb2JfdGl0bGUiOiJDbG91ZCBcdTAwMjYgQ3liZXJzZW...,NaN,Full-time,NaN,Spain,NaN,2023-08-02
3,Application Security Analyst/Consultant - DevS...,Accenture,"Madrid, Spain",via Accenture,Accenture es una compañía global de servicios ...,[{'items': ['Accenture es una compañía global ...,"[{'link': 'http://www.accenture.com/', 'text':...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[Full-time, No degree mentioned]",eyJqb2JfdGl0bGUiOiJBcHBsaWNhdGlvbiBTZWN1cml0eS...,NaN,Full-time,NaN,Spain,NaN,2023-08-02
4,Security Operations Analyst,Eventbrite,Anywhere,via Startup Jobs,THE CHALLENGE\nEventbrite is the leading globa...,[{'items': ['THE CHALLENGE Eventbrite is the l...,"[{'link': 'http://www.eventbrite.com/', 'text'...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[Work from home, Full-time, No degree mentioned]",eyJqb2JfdGl0bGUiOiJTZWN1cml0eSBPcGVyYXRpb25zIE...,NaN,Full-time,True,Spain,NaN,2023-08-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8450,SAP HCM Consultant - German and/or English Spe...,Alight Solutions LLC,"Granada, Spain",via The Muse,Integrated into the AMS department and under t...,[{'items': ['Integrated into the AMS departmen...,"[{'link': 'http://alight.com/', 'text': 'aligh...",https://encrypted-tbn0.gstatic.com/images?q=tb...,[Full-time],eyJqb2JfdGl0bGUiOiJTQVAgSENNIENvbnN1bHRhbnQgLS...,NaN,Full-time,NaN,Spain,NaN,2023-08-02
8451,Successfactors Talent Consultant,Alight,Spain (+2 others),via Careers At Alight,"Our story\n\nAt Alight, we believe a company’s...","[{'items': ['Our story At Alight, we believe ...",[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[Full-time, No degree mentioned]",eyJqb2JfdGl0bGUiOiJTdWNjZXNzZmFjdG9ycyBUYWxlbn...,NaN,Full-time,NaN,Spain,NaN,2023-08-02
8452,Associate Data & Management Consultant - Orbit...,Celonis,"Community of Madrid, Spain",via The Muse,"We're Celonis, the global leader in execution ...","[{'items': ['We're Celonis, the global leader ...","[{'link': 'http://www.celonis.de/', 'text': 'c...",https://encrypted-tbn0.gstatic.com/images?q=tb...,[Full-time],eyJqb2JfdGl0bGUiOiJBc3NvY2lhdGUgRGF0YSBcdTAwMj...,NaN,Full-time,NaN,Spain,NaN,2023-08-02
8453,UKG (Workforce dimensions) Solution Consultant...,Alight Solutions LLC,"Granada, Spain",via The Muse,"Our story\n\nAt Alight, we believe a company's...","[{'items': ['Our story At Alight, we believe ...","[{'link': 'http://alight.com/', 'text': 'aligh...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[Full-time, No degree mentioned]",eyJqb2JfdGl0bGUiOiJVS0cgKFdvcmtmb3JjZSBkaW1lbn...,NaN,Full-time,NaN,Spain,NaN,2023-08-02


In [14]:
%%time

job_types = ["Full-time", "Part-time", "Contractor", "Internship"]

# Drops Duplicates
df_new = df_new.drop_duplicates("job_id").reset_index(drop = True) 

# Cleans "via"
df_new["via"] = df_new["via"].apply(lambda x : clean.clean_source(x) if not pd.isna(x) else x)

# Cleans "location"
df_new["location"] = df_new["location"].apply(lambda x : clean.clean_location(x))

# Cleans "contract_type"
# df_new["schedule_type"] = df_new["schedule_type"].apply(lambda x : clean.clean_contract_type(x) if x not in job_types else x)

# Cleans "created_date"
df_new["posted_at"] = df_new["posted_at"].apply(lambda x : clean.transform_date(x))

df_new["date_posted"] = [clean.get_date(x, y) for x, y in df_new[["date_posted", "posted_at"]].values]

# Creates "tech_skills"
df_new["tech_skills"] = df_new["description"].apply(lambda x : clean.get_skills(x) if not pd.isna(x) else x)

CPU times: total: 36.6 s
Wall time: 1min 1s


In [15]:
df_new.to_csv("españa_03092023_segunda_ejecucion.csv", index = False)